In [12]:
import torch
from torch import nn, optim # 神经网络+优化函数
import matplotlib.pyplot as plt
from time import perf_counter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081))
])
trainset = datasets.MNIST('data', train=True, download=True, transform=transform)
testset = datasets.MNIST('data', train=False, download=True, transform=transform)

In [3]:
class LeNet(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.c1 = nn.Conv2d(1, 6, (5,5)) # connect_1: 输入1张灰度图，输出6张特征图()，5x5的卷积核过滤器
        self.c3 = nn.Conv2d(6, 16, (5,5)) # connect_3: 输入6张特征图，输出16张特征图(8*8)，5x5的卷积核过滤器
        self.fc1 = nn.Linear(16*4*4, 120) # full_connect_1: 输入为池化层S4中的所有的特征点(16*4*4)(每张池化后的特征图为4*4)，输出为120个神经元
        self.fc2 = nn.Linear(120, 84) # full_connect_2: 120->84
        self.fc3 = nn.Linear(84, 10) # full_connect_3: 84->outputs(10个数字分类0~9)

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features*=s
        return num_features

    def forward(self, x):
        x = self.c1(x) # connect_1
        x = torch.relu(x) # 激活函数-增加模型非线性拟合能力
        x = torch.max_pool2d(x, 2) # 池化层s1 池化窗口为2*2 方式为最大值池化 16*16图片池化为8*8图片

        x = self.c3(x) # connect_3
        x = torch.relu(x)
        x = torch.max_pool2d(x, 2) # 池化层s2 池化窗口为2*2 方式为最大值池化 8*8图片池化为4*4图片

        x = x.view(-1, self.num_flat_features(x)) # x由高维矩阵转化为向量形式
        x = self.fc1(x) # full_connect_1
        x = torch.relu(x)
        x = self.fc2(x) # full_connect_2
        x = torch.relu(x) 
        x = self.fc3(x) # full_connect_3 x->outputs
        return x
        


In [4]:
lenet = LeNet() # 卷积神经网络LeNet
if torch.cuda.is_available():
    lenet = lenet.cuda()

optimizer = optim.SGD(lenet.parameters(), lr=0.001, momentum=0.9) # momentum 梯度下降时的惯性 每次梯度下降优化时加上momentum*上一次梯度优化的值
criterion = nn.CrossEntropyLoss()

# batch_size 一次性加载数据量 shuffle 是否打乱 num_workers 进程数量
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

In [5]:
def train(model, criterion, optimizer, epochs=1):
    cuda = torch.cuda.is_available()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            if cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if(i%1000==999):
                print(f'[Epoch:{epoch+1}, Batch:{i+1}] Loss: {round(running_loss/1000, 3)}')
                running_loss = 0.0
    print('finished training')

In [6]:
train(lenet, criterion, optimizer, 2)

[Epoch:1, Batch:1000] Loss: 1.449
[Epoch:1, Batch:2000] Loss: 0.317
[Epoch:1, Batch:3000] Loss: 0.23
[Epoch:1, Batch:4000] Loss: 0.182
[Epoch:1, Batch:5000] Loss: 0.142
[Epoch:1, Batch:6000] Loss: 0.113
[Epoch:1, Batch:7000] Loss: 0.121
[Epoch:1, Batch:8000] Loss: 0.105
[Epoch:1, Batch:9000] Loss: 0.1
[Epoch:1, Batch:10000] Loss: 0.115
[Epoch:1, Batch:11000] Loss: 0.097
[Epoch:1, Batch:12000] Loss: 0.097
[Epoch:1, Batch:13000] Loss: 0.093
[Epoch:1, Batch:14000] Loss: 0.078
[Epoch:1, Batch:15000] Loss: 0.082
[Epoch:2, Batch:1000] Loss: 0.079
[Epoch:2, Batch:2000] Loss: 0.075
[Epoch:2, Batch:3000] Loss: 0.077
[Epoch:2, Batch:4000] Loss: 0.057
[Epoch:2, Batch:5000] Loss: 0.075
[Epoch:2, Batch:6000] Loss: 0.057
[Epoch:2, Batch:7000] Loss: 0.065
[Epoch:2, Batch:8000] Loss: 0.07
[Epoch:2, Batch:9000] Loss: 0.071
[Epoch:2, Batch:10000] Loss: 0.054
[Epoch:2, Batch:11000] Loss: 0.065
[Epoch:2, Batch:12000] Loss: 0.056
[Epoch:2, Batch:13000] Loss: 0.055
[Epoch:2, Batch:14000] Loss: 0.072
[Epoch:

In [9]:
torch.save(lenet, 'model/model.pk1') # save model

In [10]:
lenet = torch.load('model/model.pk1') # load model

In [11]:
torch.save(lenet.state_dict(), 'model/model_state.pk1') # 保存模型参数
# lenet.load_state_dict(torch.load('model/model_state.pk1')) # 保存模型参数


In [13]:
def load_param(model, path):
    if os.path.exists(path):
        model.load_state_dict(torch.load(path))

def save_param(model, path):
    torch.save(model.state_dict(), path)

In [17]:
# 测试
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

def test(testloader, model):
    correct = 0
    total = 0
    cuda = torch.cuda.is_available()
    for data in testloader:
        images, labels = data
        if cuda:
            images = images.cuda()
            labels = labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum()
    print(f'Accuracy on the test set: {100*correct/total}%')

test(testloader, lenet)

Accuracy on the test set: 98.69999694824219%


In [18]:
for data in testloader:
    print(data)
    break

[tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.42